In [22]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings

from pydantic import BaseModel

from dotenv import load_dotenv
import asyncio
import os

from IPython.display import display, Markdown
from pprint import pprint

import requests

load_dotenv(override=True)

True

# OpenAI Tools
- `WebSearchTool`: search the web
- `FileSearchTool`: retrieve from OpenAI vector stores
- `ComputerTool`: automate computer tasks

# Agents to Make
- Search Agent: search online
- Planner Agent: come up with searches based on user's query
- Report Agent: make a report
- Push Agent: send a notification to the user's phone with a summary

In [23]:
gemini_client = AsyncOpenAI(base_url=os.environ['GEMINI_BASE_URL'], api_key=os.environ['GEMINI_API_KEY'])
gemini_model = OpenAIChatCompletionsModel(model='gemini-2.5-flash-lite', openai_client=gemini_client)

#### Dummy Tools (Avoid Hosted Tools)

In [24]:
# custom dummy tools to avoid hosted tools / pricing

@function_tool
async def web_search_tool(query: str):
    """Surf the web based on the given query
    
    Args:
        query: The user's query
    """

    return """
    Search Result 1: AutoGen
    Search Result 2: LangChain
    Search Result 3: LangGraph
    Search Result 4: CrewAI
    Search Result 5: OpenAI Agents SDK
    """

#### Agent 1 - Search Agent

In [25]:
INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 words. \
Capture the main points. Write succint, no need to have complete sentences or good grammar. This will be \
consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do \
not include any additional commentary other than the summary itself."""

search_agent = Agent(
    name='Search Agent',
    instructions=INSTRUCTIONS,
    # tools=[WebSearchTool(search_context_size='low')],
    tools=[web_search_tool],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice='required'),
)

In [26]:
message = 'What are the most popular and successful AI Agent frameworks in July 2025'
with trace('Search'):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

OPENAI_API_KEY is not set, skipping trace export


As of July 2025, several AI agent frameworks are gaining significant traction and demonstrating success in the field. AutoGen, developed by Microsoft, is frequently cited for its innovative approach to multi-agent conversations, allowing for complex task orchestration and problem-solving through the collaboration of multiple AI agents. Its flexibility and ease of use have made it a popular choice for researchers and developers looking to build sophisticated AI systems.

LangChain and its extension, LangGraph, continue to be dominant forces. LangChain offers a comprehensive toolkit for developing applications powered by large language models, emphasizing modularity and composability. LangGraph builds upon this foundation, specifically focusing on enabling the creation of agents that can exhibit more complex, stateful, and cyclical behaviors. CrewAI is also emerging as a strong contender, praised for its agent-centric design that facilitates the creation of collaborative AI teams capable of executing intricate workflows. The OpenAI Agents SDK is also noteworthy, providing developers direct access to OpenAI's advanced agent capabilities, streamlining the integration of powerful AI functionalities into applications.

#### Agent 2 - Planner Agent

In [27]:
HOW_MANY_SEARCHES = 5
INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Ouptut {HOW_MANY_SEARCHES} terms to query for."

# structured outputs
class WebSearchItem(BaseModel):
    reason: str  # force the model to tell us its thinking
    "Your reasoning for why this search is important to the query"

    query: str
    "The search term to use for the web search"

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query"""

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=gemini_model,
    output_type=WebSearchPlan,
)

In [28]:
message = "What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='This is a direct query to find the most popular and successful AI agent frameworks for the specified month and year.', query='top AI agent frameworks July 2025'), WebSearchItem(reason='Understanding market share will indicate which frameworks are most adopted and likely considered successful.', query='AI agent framework market share 2025'), WebSearchItem(reason='Usage statistics provide concrete data on the popularity and adoption rates of different frameworks.', query='AI agent framework usage statistics 2025'), WebSearchItem(reason='This search aims to identify platforms that are built upon or integrate popular AI agent frameworks, indirectly indicating framework success.', query='most used AI agent platforms 2025'), WebSearchItem(reason='Benchmarks can highlight performance and capabilities, which are key factors in determining the success of a framework.', query='AI agent framework benchmarks 2025')])


In [29]:
for search in result.final_output.searches:
    print(search.reason)
    print(search.query)
    print()

This is a direct query to find the most popular and successful AI agent frameworks for the specified month and year.
top AI agent frameworks July 2025

Understanding market share will indicate which frameworks are most adopted and likely considered successful.
AI agent framework market share 2025

Usage statistics provide concrete data on the popularity and adoption rates of different frameworks.
AI agent framework usage statistics 2025

This search aims to identify platforms that are built upon or integrate popular AI agent frameworks, indirectly indicating framework success.
most used AI agent platforms 2025

Benchmarks can highlight performance and capabilities, which are key factors in determining the success of a framework.
AI agent framework benchmarks 2025



#### Agent 3 - Writer Agent

In [30]:
INSTRUCTIONS = """
You are a senior researcher tasked with writing a cohesive report for a research query.
You will be provided with the original query and some initial research done by a research assistant.

You should first come up with an outline for the report that describes the structure and flow of the report.
Then, generate the report and return that as your final output.

The final output should be in markdown format and it should be length and detailed.
Aim for 5-10 pages of content, at least 1000 words.
"""

class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings"""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

writer_agent = Agent(
    name='WriterAgent',
    instructions=INSTRUCTIONS,
    model=gemini_model,
    output_type=ReportData
)

#### Agent 4 - Push Notification Agent

In [31]:
# https://pushover.net

pushover_user = ...
pushover_token = ...
pushover_url = ...

@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

# dummy tool to avoid tools with pricing
@function_tool
def push_tool(message: str):
    """Send a push notification with this brief message"""
    print(f'Dummy tool called to send push notification with message: {message}')
    return {"status": "success"}

In [32]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you received."""

push_agent = Agent(
    name='push_agent',
    instructions=INSTRUCTIONS,
    tools=[push_tool],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice=['required'])
)

#### Plan and Execute Functions

In [33]:
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    
    print('Planning searches...')
    result = await Runner.run(planner_agent, f'Query: {query}')
    
    print(f'Will perform {len(result.final_output.searches)} searches')
    
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan"""
    
    print('Searching...')
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    
    print('Finished searching')
    return results

async def search(item: WebSearchItem):
    """Use the search_agent to run a web search for each item in the search plan"""

    input_ = f'Search term: {item.query}\nReason for searching: {item.reason}'
    result = await Runner.run(search_agent, input_)

    return result.final_output


#### Report and Notification Functions

In [34]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer_agent to write a report based on the search results"""

    print('Thinking about the report...')
    input_ = f'Original query: {query}\nSummarized search results: {search_results}'
    result = await Runner.run(writer_agent, input_)
    
    print('Finished writing')
    return result.final_output

async def send_push(report: ReportData):
    """Use the push_agent to send a notification to the user"""

    print('Pushing...')
    _ = await Runner.run(push_agent, report.short_summary)
    
    print('Push sent')
    return report

In [ ]:
query = 'What are the most popular and successful AI Agent frameworks in July 2025'

with trace('Research trace'):
    print('Starting research...')

    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)

    print('Done!')

display(Markdown(report.markdown_report))